# NFL Game Prediction Model

This notebook builds a machine learning model to predict NFL game outcomes using historical team performance data. The model analyzes team statistics like passing yards, rushing yards, points scored, and other key metrics to make predictions.

## Overview
1. **Setup & Dependencies** - Install required packages and import libraries
2. **Data Collection** - Gather NFL data from multiple seasons using nfl_data_py
3. **Model Training** - Train ensemble model using multiple algorithms
4. **Game Prediction Functions** - Create tools to predict individual or multiple games
5. **Week 1 Predictions** - Generate predictions for all NFL Week 1 games

## Step 1: Setup & Dependencies

Installing the required packages:
- **xgboost**: Advanced gradient boosting machine learning library
- **nfl_data_py**: Official NFL data package for accessing play-by-play, team stats, and schedule data

In [11]:
pip install xgboost nfl_data_py

Note: you may need to restart the kernel to use updated packages.


## Step 2: NFLGamePredictor Class & Data Collection

This cell defines the main `NFLGamePredictor` class that handles:

**Data Collection:**
- Downloads 10+ years of NFL data (2015-2025)
- Collects play-by-play data, team statistics, and game schedules
- Processes team performance metrics (passing/rushing yards, points, turnovers)

**Feature Engineering:**
- Creates team-level features from raw game data
- Calculates advantages between home/away teams
- Includes contextual factors like home field advantage

**Model Training:**
- Tests multiple algorithms (Random Forest, XGBoost, Logistic Regression)
- Uses feature selection to identify most predictive statistics
- Creates ensemble model combining best performers

In [12]:
# NFL Game Prediction using nfl_data_py
# Updated version of NFL prediction pipeline using modern, maintained data source

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

# Core ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.calibration import CalibratedClassifierCV as CCV
from sklearn.pipeline import Pipeline

# Advanced ML libraries
try:
    import xgboost as xgb
    HAS_XGB = True
except ImportError:
    HAS_XGB = False
    print("XGBoost not available. Install with: pip install xgboost")

# NFL data library
try:
    import nfl_data_py as nfl
    HAS_NFL_DATA = True
except ImportError:
    HAS_NFL_DATA = False
    print("nfl_data_py not available. Install with: pip install nfl_data_py")

class NFLGamePredictor:
    def __init__(self):
        self.models = {}
        self.best_features = []
        self.scaler = StandardScaler()
        self.final_model = None
        
    def collect_data(self, start_year=2010, end_year=2024, save_csv=True, data_folder='nfl_data'):
        """Collect comprehensive NFL data using nfl_data_py and save as CSV files"""
        
        if not HAS_NFL_DATA:
            raise ImportError("nfl_data_py is required. Install with: pip install nfl_data_py")
        
        print(f"Collecting NFL data from {start_year} to {end_year}...")
        
        # Create data folder if it doesn't exist
        if save_csv and not os.path.exists(data_folder):
            os.makedirs(data_folder)
            print(f"Created data folder: {data_folder}")
        
        years = list(range(start_year, end_year + 1))
        
        # Get play-by-play data for game-level statistics
        print("Downloading play-by-play data...")
        try:
            pbp_data = nfl.import_pbp_data(years)
            print(f"✓ Play-by-play data downloaded successfully")
        except Exception as e:
            print(f"Error downloading play-by-play data: {e}")
            # Try with reduced year range
            years = list(range(start_year, 2024 + 1))
            pbp_data = nfl.import_pbp_data(years)
            print(f"✓ Play-by-play data downloaded (adjusted to {start_year}-2024)")
        
        # Get team information
        print("Downloading team information...")
        teams = nfl.import_team_desc()
        team_dict = teams.set_index('team_abbr')['team_name'].to_dict()
        
        # Get weekly data for season-long team statistics
        print("Downloading weekly team data...")
        try:
            weekly_data = nfl.import_weekly_data(years)
            print(f"✓ Weekly data downloaded successfully")
        except Exception as e:
            print(f"Error downloading weekly data: {e}")
            print("Trying with 2015-2024 data only...")
            # Fall back to confirmed available years
            years_safe = list(range(2015, 2024 + 1))
            weekly_data = nfl.import_weekly_data(years_safe)
            print(f"✓ Weekly data downloaded (2015-2024)")
        
        # Get schedule data
        print("Downloading schedule data...")
        try:
            schedule_data = nfl.import_schedules(years)
            print(f"✓ Schedule data downloaded successfully")
        except Exception as e:
            print(f"Error downloading schedule data: {e}")
            print("Trying with 2015-2024 data only...")
            # Fall back to confirmed available years
            years_safe = list(range(2015, 2024 + 1))
            schedule_data = nfl.import_schedules(years_safe)
            print(f"✓ Schedule data downloaded (2015-2024)")
        
        # Try to get 2025 Week 1 data separately if available
        print("Attempting to download 2025 Week 1 data...")
        try:
            pbp_2025 = nfl.import_pbp_data([2025])
            if not pbp_2025.empty:
                pbp_data = pd.concat([pbp_data, pbp_2025], ignore_index=True)
                print("✓ 2025 play-by-play data added")
            
            weekly_2025 = nfl.import_weekly_data([2025])
            if not weekly_2025.empty:
                weekly_data = pd.concat([weekly_data, weekly_2025], ignore_index=True)
                print("✓ 2025 weekly data added")
                
            schedule_2025 = nfl.import_schedules([2025])
            if not schedule_2025.empty:
                schedule_data = pd.concat([schedule_data, schedule_2025], ignore_index=True)
                print("✓ 2025 schedule data added")
        except Exception as e:
            print(f"2025 data not available yet: {e}")
            print("Proceeding with 2015-2024 data only")
        
        print(f"Collected {len(pbp_data)} play-by-play records")
        print(f"Collected {len(weekly_data)} weekly team records") 
        print(f"Collected {len(schedule_data)} scheduled games")
        
        # Save data as CSV files if requested
        if save_csv:
            print(f"\nSaving data to CSV files in '{data_folder}' folder...")
            
            # Save play-by-play data (this will be large)
            pbp_file = os.path.join(data_folder, f'pbp_data_{start_year}_{end_year}.csv')
            pbp_data.to_csv(pbp_file, index=False)
            print(f"✓ Saved play-by-play data: {pbp_file} ({len(pbp_data):,} rows)")
            
            # Save weekly data
            weekly_file = os.path.join(data_folder, f'weekly_data_{start_year}_{end_year}.csv')
            weekly_data.to_csv(weekly_file, index=False)
            print(f"✓ Saved weekly data: {weekly_file} ({len(weekly_data):,} rows)")
            
            # Save schedule data
            schedule_file = os.path.join(data_folder, f'schedule_data_{start_year}_{end_year}.csv')
            schedule_data.to_csv(schedule_file, index=False)
            print(f"✓ Saved schedule data: {schedule_file} ({len(schedule_data):,} rows)")
            
            # Save team information
            teams_file = os.path.join(data_folder, 'team_info.csv')
            teams.to_csv(teams_file, index=False)
            print(f"✓ Saved team info: {teams_file} ({len(teams):,} rows)")
            
            # Save a sample of each dataset for quick inspection
            print(f"\nSaving sample data for quick inspection...")
            
            # Sample play-by-play (first 1000 rows)
            pbp_sample_file = os.path.join(data_folder, 'pbp_sample.csv')
            pbp_data.head(1000).to_csv(pbp_sample_file, index=False)
            print(f"✓ Saved PBP sample: {pbp_sample_file} (1,000 rows)")
            
            # Sample weekly data (recent season)
            weekly_sample = weekly_data[weekly_data['season'] >= end_year - 1]
            weekly_sample_file = os.path.join(data_folder, 'weekly_sample.csv')
            weekly_sample.to_csv(weekly_sample_file, index=False)
            print(f"✓ Saved weekly sample: {weekly_sample_file} ({len(weekly_sample):,} rows from {end_year-1}-{end_year})")
            
            # Show column information
            print(f"\nDATA STRUCTURE OVERVIEW:")
            print("="*50)
            print(f"Play-by-Play Columns ({len(pbp_data.columns)}): {list(pbp_data.columns[:10])}...")
            print(f"Weekly Data Columns ({len(weekly_data.columns)}): {list(weekly_data.columns)}")
            print(f"Schedule Columns ({len(schedule_data.columns)}): {list(schedule_data.columns)}")
            print(f"Team Info Columns ({len(teams.columns)}): {list(teams.columns)}")
            
            print(f"\nDATA SAVED SUCCESSFULLY!")
            print(f"Check the '{data_folder}' folder to examine the downloaded data.")
        
        return pbp_data, weekly_data, schedule_data, team_dict
    
    def _calculate_injury_percentage(self, team_data):
        """
        Calculate estimated injury percentage based on available performance metrics
        
        This method estimates team injury impact by analyzing performance consistency
        and key player availability indicators in the data.
        """
        
        # If we have specific injury data columns, use them
        if 'injuries' in team_data.columns:
            return team_data['injuries'].mean()
        
        # Estimate injury impact based on performance variance and available metrics
        # Higher variance in key stats might indicate injury-related inconsistency
        
        injury_indicators = []
        
        # 1. Passing performance consistency (QB health indicator)
        if 'passing_yards' in team_data.columns and len(team_data) > 1:
            passing_std = team_data['passing_yards'].std()
            passing_mean = team_data['passing_yards'].mean()
            if passing_mean > 0:
                passing_variance = (passing_std / passing_mean) * 100
                injury_indicators.append(min(passing_variance, 30))  # Cap at 30%
        
        # 2. Rushing performance consistency (RB/OL health indicator)
        if 'rushing_yards' in team_data.columns and len(team_data) > 1:
            rushing_std = team_data['rushing_yards'].std()
            rushing_mean = team_data['rushing_yards'].mean()
            if rushing_mean > 0:
                rushing_variance = (rushing_std / rushing_mean) * 100
                injury_indicators.append(min(rushing_variance, 25))  # Cap at 25%
        
        # 3. Completion percentage consistency (QB/WR health indicator)
        if 'completions' in team_data.columns and 'passing_attempts' in team_data.columns:
            comp_pct = team_data['completions'] / (team_data['passing_attempts'] + 0.1)  # Avoid division by zero
            if len(comp_pct) > 1:
                comp_std = comp_pct.std()
                comp_variance = comp_std * 100
                injury_indicators.append(min(comp_variance, 20))  # Cap at 20%
        
        # 4. Fantasy points consistency (overall team health)
        if 'fantasy_points' in team_data.columns and len(team_data) > 1:
            fp_std = team_data['fantasy_points'].std()
            fp_mean = team_data['fantasy_points'].mean()
            if fp_mean > 0:
                fp_variance = (fp_std / fp_mean) * 100
                injury_indicators.append(min(fp_variance, 35))  # Cap at 35%
        
        # Calculate weighted average injury percentage
        if injury_indicators:
            # Weight more recent games higher (if we have game order info)
            weights = [1.0] * len(injury_indicators)  
            weighted_avg = sum(i * w for i, w in zip(injury_indicators, weights)) / sum(weights)
            
            # Normalize to 0-100% range and apply league baseline
            # NFL teams typically have 10-25% of roster dealing with some injury
            baseline_injury_rate = 15.0  # League average baseline
            estimated_injury_pct = min(max(weighted_avg, 5.0), 40.0)  # 5-40% range
            
            # Blend with baseline for more realistic estimates
            final_injury_pct = (estimated_injury_pct * 0.7) + (baseline_injury_rate * 0.3)
            
            return final_injury_pct
        
        # Default injury rate if no data available
        return 15.0  # NFL league average
    
    def create_team_features(self, weekly_data, season, week):
        """Create team-level features for a specific season/week"""
        
        # Filter data up to the current week
        season_data = weekly_data[
            (weekly_data['season'] == season) & 
            (weekly_data['week'] < week)
        ]
        
        if season_data.empty:
            return {}
        
        # Calculate season averages for each team
        team_features = {}
        
        for team in season_data['recent_team'].unique():
            team_data = season_data[season_data['recent_team'] == team]
            
            if len(team_data) == 0:
                continue
                
            # Offensive features
            features = {
                'passing_yards_pg': team_data['passing_yards'].mean(),
                'rushing_yards_pg': team_data['rushing_yards'].mean(), 
                'total_yards_pg': team_data['passing_yards'].mean() + team_data['rushing_yards'].mean(),
                'points_pg': team_data['fantasy_points'].mean() if 'fantasy_points' in team_data.columns else 0,
                'completions_pg': team_data['completions'].mean(),
                'passing_tds_pg': team_data['passing_tds'].mean(),
                'interceptions_thrown_pg': team_data['interceptions'].mean(),
                'rushing_tds_pg': team_data['rushing_tds'].mean(),
                'fumbles_lost_pg': team_data['fumbles_lost'].mean() if 'fumbles_lost' in team_data.columns else 0,
                
                # Defensive features (opponent stats)
                'opp_passing_yards_pg': 0,  # Will be calculated separately
                'opp_rushing_yards_pg': 0,
                'opp_points_pg': 0,
                
                # Team health and availability metrics
                'injury_percentage': self._calculate_injury_percentage(team_data),
                
                # Advanced metrics
                'turnover_ratio': (team_data['interceptions'].mean() - 
                                 team_data['interceptions'].mean()),  # Simplified
                'games_played': len(team_data)
            }
            
            team_features[team] = features
        
        return team_features
    
    def create_game_features(self, home_team, away_team, team_features, 
                           season, week, is_playoff=False, is_neutral=False):
        """Create features for a specific matchup"""
        
        if home_team not in team_features or away_team not in team_features:
            return None
        
        home_stats = team_features[home_team]
        away_stats = team_features[away_team]
        
        # Create matchup features
        features = {
            # Home team offensive stats
            'home_passing_ypg': home_stats['passing_yards_pg'],
            'home_rushing_ypg': home_stats['rushing_yards_pg'],
            'home_total_ypg': home_stats['total_yards_pg'],
            'home_points_pg': home_stats['points_pg'],
            'home_passing_tds_pg': home_stats['passing_tds_pg'],
            'home_turnovers_pg': home_stats.get('fumbles_lost_pg', 0) + home_stats['interceptions_thrown_pg'],
            'home_injury_pct': home_stats['injury_percentage'],
            
            # Away team offensive stats  
            'away_passing_ypg': away_stats['passing_yards_pg'],
            'away_rushing_ypg': away_stats['rushing_yards_pg'],
            'away_total_ypg': away_stats['total_yards_pg'],
            'away_points_pg': away_stats['points_pg'],
            'away_passing_tds_pg': away_stats['passing_tds_pg'],
            'away_turnovers_pg': away_stats.get('fumbles_lost_pg', 0) + away_stats['interceptions_thrown_pg'],
            'away_injury_pct': away_stats['injury_percentage'],
            
            # Matchup advantages
            'passing_advantage': home_stats['passing_yards_pg'] - away_stats['passing_yards_pg'],
            'rushing_advantage': home_stats['rushing_yards_pg'] - away_stats['rushing_yards_pg'],
            'scoring_advantage': home_stats['points_pg'] - away_stats['points_pg'],
            'turnover_advantage': away_stats.get('fumbles_lost_pg', 0) + away_stats['interceptions_thrown_pg'] - 
                                (home_stats.get('fumbles_lost_pg', 0) + home_stats['interceptions_thrown_pg']),
            'injury_advantage': away_stats['injury_percentage'] - home_stats['injury_percentage'],  # Lower injury % is better
            
            # Game context
            'home_field_advantage': 0 if is_neutral else 2.5,
            'is_playoff': 1 if is_playoff else 0,
            'is_neutral': 1 if is_neutral else 0,
            'week': week,
            'season': season,
        }
        
        return features
    
    def build_dataset(self, pbp_data, weekly_data, schedule_data, save_csv=True, data_folder='nfl_data'):
        """Build complete dataset from NFL data and optionally save as CSV"""
        
        print("Building dataset from collected data...")
        
        game_records = []
        
        # Process each scheduled game
        for _, game in schedule_data.iterrows():
            season = game['season']
            week = game['week']
            home_team = game['home_team']
            away_team = game['away_team']
            
            # Skip if missing essential data
            if pd.isna(home_team) or pd.isna(away_team):
                continue
            
            # Get team features up to this point in season
            team_features = self.create_team_features(weekly_data, season, week)
            
            if not team_features:
                continue
            
            # Create game features
            game_features = self.create_game_features(
                home_team, away_team, team_features, 
                season, week,
                is_playoff=game.get('game_type', '') == 'REG',
                is_neutral=False  # Simplified for now
            )
            
            if game_features is None:
                continue
            
            # Determine result (home team win = 1, loss = 0)
            home_score = game.get('home_score', 0)
            away_score = game.get('away_score', 0)
            
            # Skip games without scores (future games)
            if pd.isna(home_score) or pd.isna(away_score):
                continue
            
            game_features['home_win'] = 1 if home_score > away_score else 0
            game_features['home_score'] = home_score
            game_features['away_score'] = away_score
            game_features['game_id'] = f"{season}_{week}_{home_team}_{away_team}"
            
            game_records.append(game_features)
        
        df = pd.DataFrame(game_records)
        print(f"Created dataset with {len(df)} games")
        
        # Save the processed dataset
        if save_csv and not df.empty:
            if not os.path.exists(data_folder):
                os.makedirs(data_folder)
            
            processed_file = os.path.join(data_folder, 'processed_game_features.csv')
            df.to_csv(processed_file, index=False)
            print(f"✓ Saved processed game features: {processed_file} ({len(df):,} rows)")
            
            # Show feature information
            print(f"\nPROCESSED FEATURES ({len(df.columns)} columns):")
            print("="*50)
            for col in df.columns:
                print(f"  - {col}")
        
        return df
    
    def select_features(self, df, n_features=13):
        """Select best features using RFE"""
        
        # Prepare data
        feature_cols = [col for col in df.columns 
                       if col not in ['home_win', 'home_score', 'away_score', 'game_id']]
        
        X = df[feature_cols]
        y = df['home_win']
        
        # Remove any columns with all NaN or constant values
        X = X.loc[:, X.var() > 0]
        X = X.fillna(X.mean())
        
        print(f"Starting feature selection with {len(X.columns)} features...")
        
        # Use RFE with different numbers of features
        models = {}
        results = []
        
        for i in range(2, min(n_features + 1, len(X.columns) + 1)):
            rfe = RFE(estimator=LDA(), n_features_to_select=i)
            model = DecisionTreeClassifier(random_state=42)
            pipeline = Pipeline(steps=[('s', rfe), ('m', model)])
            
            cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
            scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
            
            results.append(scores)
            models[str(i)] = pipeline
            
            print(f'{i} features: {scores.mean():.3f} (+/- {scores.std():.3f})')
        
        # Find best number of features
        best_idx = np.argmax([np.mean(result) for result in results])
        best_n_features = best_idx + 2
        
        print(f"Best number of features: {best_n_features}")
        
        # Get the best feature set
        rfe = RFE(estimator=LDA(), n_features_to_select=best_n_features)
        rfe.fit(X, y)
        
        selected_features = X.columns[rfe.support_].tolist()
        self.best_features = selected_features
        
        print("Selected features:")
        for feature in selected_features:
            print(f"  - {feature}")
        
        return selected_features
    
    def train_models(self, df):
        """Train and compare multiple models"""
        
        if not self.best_features:
            self.select_features(df)
        
        # Prepare data
        X = df[self.best_features].fillna(df[self.best_features].mean())
        y = df['home_win']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Define models to test
        models_to_test = {
            'Logistic Regression': LogisticRegression(random_state=42),
            'Decision Tree': DecisionTreeClassifier(random_state=42),
            'Random Forest': RandomForestClassifier(random_state=42)
        }
        
        if HAS_XGB:
            models_to_test['XGBoost'] = xgb.XGBClassifier(random_state=42, verbosity=0)
        
        # Train and evaluate models
        model_results = {}
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
        
        print("\nTraining and evaluating models...")
        
        for name, model in models_to_test.items():
            # Use scaled data for logistic regression, raw for tree-based
            X_use = X_train_scaled if 'Logistic' in name else X_train
            scores = cross_val_score(model, X_use, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
            
            model_results[name] = {
                'mean_score': scores.mean(),
                'std_score': scores.std(),
                'scores': scores
            }
            
            print(f"{name}: {scores.mean():.3f} (+/- {scores.std():.3f})")
        
        self.models = models_to_test
        return model_results
    
    def tune_best_model(self, df, model_name='Random Forest'):
        """Tune hyperparameters for the best performing model"""
        
        X = df[self.best_features].fillna(df[self.best_features].mean())
        y = df['home_win']
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        if model_name == 'Random Forest':
            param_grid = {
                'n_estimators': [200, 300, 400, 500],
                'max_depth': [8, 9, 10, 11, 12],
                'min_samples_leaf': [2, 5],
                'criterion': ['gini', 'entropy']
            }
            model = RandomForestClassifier(random_state=42)
        
        elif model_name == 'Logistic Regression':
            param_grid = {
                'C': [0.01, 0.1, 1, 10, 100],
                'penalty': ['l1', 'l2'],
                'solver': ['liblinear']
            }
            model = LogisticRegression(random_state=42)
            X_train = self.scaler.fit_transform(X_train)
            X_test = self.scaler.transform(X_test)
        
        elif model_name == 'XGBoost' and HAS_XGB:
            param_grid = {
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'n_estimators': [100, 200, 300],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'reg_alpha': [0, 1],
            'reg_lambda': [1, 5]
            }
            model = xgb.XGBClassifier(random_state=42, verbosity=0)
        
        else:
            print(f"Tuning not implemented for {model_name}")
            return None
        
        print(f"\nTuning {model_name}...")
        
        # Grid search
        grid_search = GridSearchCV(
            model, param_grid, 
            cv=5, scoring='accuracy', 
            n_jobs=-1, verbose=1
        )
        
        grid_search.fit(X_train, y_train)
        
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Best cross-validation score: {grid_search.best_score_:.3f}")
        
        # Test on holdout set
        test_score = grid_search.score(X_test, y_test)
        print(f"Test set accuracy: {test_score:.3f}")
        
        return grid_search.best_estimator_
    
    def create_ensemble_model(self, df):
        """Create ensemble model combining multiple algorithms"""
        
        X = df[self.best_features].fillna(df[self.best_features].mean())
        y = df['home_win']
        
        # Individual models
        rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
        lr = LogisticRegression(C=1, random_state=42)
        
        estimators = [('rf', rf), ('lr', lr)]
        
        if HAS_XGB:
            xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=200, 
                                        random_state=42, verbosity=0)
            estimators.append(('xgb', xgb_model))
        
        # Voting classifier
        voting_clf = VotingClassifier(estimators=estimators, voting='soft')
        
        # Calibrated classifier for better probability estimates
        self.final_model = CCV(voting_clf, method='isotonic', cv=3)
        
        print("Training ensemble model...")
        self.final_model.fit(X, y)
        
        return self.final_model
    
    def predict_games(self, games_df, confidence_threshold=0.6):
        """Make predictions on new games"""
        
        if self.final_model is None:
            raise ValueError("Model not trained yet. Call create_ensemble_model() first.")
        
        X = games_df[self.best_features].fillna(games_df[self.best_features].mean())
        
        # Get probability predictions
        probabilities = self.final_model.predict_proba(X)[:, 1]
        predictions = self.final_model.predict(X)
        
        # Create results dataframe
        results = games_df.copy()
        results['home_win_prob'] = probabilities
        results['predicted_home_win'] = predictions
        
        # High-confidence bets
        results['high_confidence_bet'] = (
            (probabilities >= confidence_threshold) | 
            (probabilities <= (1 - confidence_threshold))
        )
        
        return results

# Example usage and testing
if __name__ == "__main__":
    
    # Initialize predictor
    predictor = NFLGamePredictor()
    
    # Check if nfl_data_py is available
    if not HAS_NFL_DATA:
        print("Please install nfl_data_py to use this predictor:")
        print("pip install nfl_data_py")
        exit()
    
    try:
        # Collect data (this may take a few minutes)
        print("This may take a few minutes to download NFL data...")
        pbp_data, weekly_data, schedule_data, team_dict = predictor.collect_data(2015, 2025, save_csv=True)
        
        # Build dataset
        df = predictor.build_dataset(pbp_data, weekly_data, schedule_data, save_csv=True)
        
        if df.empty:
            print("No data collected. Check your internet connection and try again.")
            exit()
        
        # Feature selection
        predictor.select_features(df, n_features=13)
        
        # Train models
        model_results = predictor.train_models(df)
        
        # Create ensemble
        final_model = predictor.create_ensemble_model(df)
        
        # Example prediction on test set
        train_data = df[df['season'] < 2023]
        test_data = df[df['season'] == 2023]
        
        if not test_data.empty:
            predictor.final_model.fit(
                train_data[predictor.best_features].fillna(train_data[predictor.best_features].mean()), 
                train_data['home_win']
            )
            
            predictions = predictor.predict_games(test_data)
            
            # Calculate accuracy
            accuracy = (predictions['predicted_home_win'] == predictions['home_win']).mean()
            print(f"\n2023 season prediction accuracy: {accuracy:.3f}")
            
            # High confidence bets
            high_conf = predictions[predictions['high_confidence_bet']]
            if not high_conf.empty:
                conf_accuracy = (high_conf['predicted_home_win'] == high_conf['home_win']).mean()
                print(f"High confidence bet accuracy: {conf_accuracy:.3f} ({len(high_conf)} games)")
        
        print("\nPredictor trained successfully!")
        print("You can now use predictor.predict_games() on new data.")
        
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have a stable internet connection for data download.")

This may take a few minutes to download NFL data...
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
2025 done.
Downcasting floats.
✓ Play-by-play data downloaded successfully
Error downloading weekly data: HTTP Error 404: Not Found
Trying with 2015-2024 data only...
Downcasting floats.
✓ Weekly data downloaded (2015-2024)
✓ Schedule data downloaded successfully
Attempting to download 2025 Week 1 data...
2025 done.
Downcasting floats.
✓ 2025 play-by-play data added
2025 data not available yet: HTTP Error 404: Not Found
Proceeding with 2015-2024 data only
Collected 498057 play-by-play records
Collected 54479 weekly team records
Collected 3015 scheduled games

Saving data to CSV files in 'nfl_data' folder...
✓ Saved play-by-play data: nfl_data\pbp_data_2015_2025.csv (498,057 rows)
✓ Saved weekly data: nfl_data\weekly_data_2015_2025.csv (54,479 rows)
✓ Saved schedule data: nfl_data\schedule_data_2015_2025.csv (3,015 rows)
✓ Save

## Step 3: Save the Model (Optional)


In [13]:
import joblib

# Save the trained model

joblib.dump(predictor.final_model, 'final_model.joblib')

# To load later:
# predictor.final_model = joblib.load('nfl_data/final_model.joblib')

['final_model.joblib']

## Step 4: Individual Game Prediction

This function allows you to predict the outcome of any specific matchup by providing:
- Home team abbreviation (e.g., 'NYJ' for Jets)
- Away team abbreviation (e.g., 'PIT' for Steelers)
- Season and week

**Output includes:**
- Predicted winner
- Confidence percentage
- Win probabilities for both teams

In [ ]:
# Make predictions for specific games
def predict_specific_game(predictor, home_team, away_team, season=2025, week=1):
    """
    Predict the outcome of a specific game
    
    Args:
        predictor: Trained NFLGamePredictor instance
        home_team: Home team abbreviation (e.g., 'NYJ')
        away_team: Away team abbreviation (e.g., 'PIT')
        season: Season year
        week: Week number
    """
    
    # Get 2024 team features to predict 2025 games
    team_features = predictor.create_team_features(weekly_data, 2024, 19)  # End of 2024 season
    
    if not team_features or home_team not in team_features or away_team not in team_features:
        print(f"Error: Could not find team data for {home_team} or {away_team}")
        return None
    
    # Create game features
    game_features = predictor.create_game_features(
        home_team, away_team, team_features, 
        season, week,
        is_playoff=False,
        is_neutral=False
    )
    
    if game_features is None:
        print("Error: Could not create game features")
        return None
    
    # Convert to DataFrame for prediction
    game_df = pd.DataFrame([game_features])
    
    # Make prediction
    prediction_result = predictor.predict_games(game_df)
    
    # Extract results
    home_win_prob = prediction_result['home_win_prob'].iloc[0]
    predicted_home_win = prediction_result['predicted_home_win'].iloc[0]
    high_confidence = prediction_result['high_confidence_bet'].iloc[0]
    
    # Display results
    winner = home_team if predicted_home_win == 1 else away_team
    confidence = home_win_prob if predicted_home_win == 1 else (1 - home_win_prob)
    
    print(f"\n🏈 GAME PREDICTION: {away_team} @ {home_team}")
    print(f"{'='*50}")
    print(f"🏆 Predicted Winner: {winner}")
    print(f"🎯 Confidence: {confidence:.1%}")
    print(f"📊 Home Win Probability: {home_win_prob:.1%}")
    print(f"📊 Away Win Probability: {(1-home_win_prob):.1%}")
    print(f"🔥 High Confidence Bet: {'Yes' if high_confidence else 'No'}")
    
    return prediction_result

# Example: Predict Upcoming Games
if 'predictor' in locals() and predictor.final_model is not None:
    steelers_jets_prediction = predict_specific_game(
        predictor, 
        home_team='LAC',  
        away_team='KC',  
        season=2025, 
        week=1
    )
else:
    print("Please run the training cell first to create the predictor model")


🏈 GAME PREDICTION: KC @ LAC
🏆 Predicted Winner: LAC
🎯 Confidence: 51.5%
📊 Home Win Probability: 51.5%
📊 Away Win Probability: 48.5%
🔥 High Confidence Bet: No


## Step 5: Multiple Game Prediction System

This advanced function processes entire game slates and includes:

**Features:**
- Batch prediction for multiple games
- Team name mapping and validation
- Game scheduling with times and dates
- Chronological ordering of results

**Input Format:** 
```
(Away) Team Name vs. (Home) Team Name
```

**Outputs:**
- Individual game results with times
- Summary statistics
- Top confident predictions

In [35]:
# MULTIPLE GAME PREDICTIONS - Predict entire game slates
import re
from datetime import datetime

def predict_multiple_games(predictor, games_text, season=2025, week=2, show_details=True):
    """
    Predict multiple games from formatted text
    
    Args:
        predictor: Trained NFLGamePredictor instance
        games_text: Formatted text with games (Away) TEAM vs. (Home) TEAM
        season: Season year
        week: Week number
        show_details: Whether to show detailed output for each game
    
    Returns:
        DataFrame with all predictions
    """
    
    if predictor.final_model is None:
        print("Model not trained yet. Please train the model first.")
        return None
    
    # Team abbreviation mapping for common variations
    team_mapping = {
        'TB Buccaneers': 'TB', 'Tampa Bay Buccaneers': 'TB', 'TB': 'TB',
        'ATL Falcons': 'ATL', 'Atlanta Falcons': 'ATL', 'ATL': 'ATL',
        'CIN Bengals': 'CIN', 'Cincinnati Bengals': 'CIN', 'CIN': 'CIN',
        'CLE Browns': 'CLE', 'Cleveland Browns': 'CLE', 'CLE': 'CLE',
        'MIA Dolphins': 'MIA', 'Miami Dolphins': 'MIA', 'MIA': 'MIA',
        'IND Colts': 'IND', 'Indianapolis Colts': 'IND', 'IND': 'IND',
        'CAR Panthers': 'CAR', 'Carolina Panthers': 'CAR', 'CAR': 'CAR',
        'JAX Jaguars': 'JAX', 'Jacksonville Jaguars': 'JAX', 'JAX': 'JAX',
        'LV Raiders': 'LV', 'Las Vegas Raiders': 'LV', 'LV': 'LV',
        'NE Patriots': 'NE', 'New England Patriots': 'NE', 'NE': 'NE',
        'ARI Cardinals': 'ARI', 'Arizona Cardinals': 'ARI', 'ARI': 'ARI',
        'NO Saints': 'NO', 'New Orleans Saints': 'NO', 'NO': 'NO',
        'PIT Steelers': 'PIT', 'Pittsburgh Steelers': 'PIT', 'PIT': 'PIT',
        'NYJ Jets': 'NYJ', 'New York Jets': 'NYJ', 'NYJ': 'NYJ',
        'NYG Giants': 'NYG', 'New York Giants': 'NYG', 'NYG': 'NYG',
        'WAS Commanders': 'WAS', 'Washington Commanders': 'WAS', 'WAS': 'WAS',
        'TEN Titans': 'TEN', 'Tennessee Titans': 'TEN', 'TEN': 'TEN',
        'DEN Broncos': 'DEN', 'Denver Broncos': 'DEN', 'DEN': 'DEN',
        'SF 49ers': 'SF', 'San Francisco 49ers': 'SF', 'SF': 'SF',
        'SEA Seahawks': 'SEA', 'Seattle Seahawks': 'SEA', 'SEA': 'SEA',
        'DET Lions': 'DET', 'Detroit Lions': 'DET', 'DET': 'DET',
        'GB Packers': 'GB', 'Green Bay Packers': 'GB', 'GB': 'GB',
        'HOU Texans': 'HOU', 'Houston Texans': 'HOU', 'HOU': 'HOU',
        'LA Rams': 'LA', 'Los Angeles Rams': 'LA', 'LA': 'LA',
        'LAR Rams': 'LA', 'Los Angeles Rams': 'LA', 'LAR': 'LA',  # Handle LAR variation
        'BAL Ravens': 'BAL', 'Baltimore Ravens': 'BAL', 'BAL': 'BAL',
        'BUF Bills': 'BUF', 'Buffalo Bills': 'BUF', 'BUF': 'BUF',
        'KC Chiefs': 'KC', 'Kansas City Chiefs': 'KC', 'KC': 'KC',
        'PHI Eagles': 'PHI', 'Philadelphia Eagles': 'PHI', 'PHI': 'PHI',
        'DAL Cowboys': 'DAL', 'Dallas Cowboys': 'DAL', 'DAL': 'DAL',
        'MIN Vikings': 'MIN', 'Minnesota Vikings': 'MIN', 'MIN': 'MIN',
        'CHI Bears': 'CHI', 'Chicago Bears': 'CHI', 'CHI': 'CHI',
        'LAC Chargers': 'LAC', 'Los Angeles Chargers': 'LAC', 'LAC': 'LAC'
    }
    
    
    # Parse games from text
    games = []
    game_pattern = r'\(Away\)\s+(.*?)\s+vs\.\s+\(Home\)\s+(.*)'
    
    matches = re.findall(game_pattern, games_text, re.IGNORECASE)
    
    if not matches:
        print("No games found in the provided text. Expected format:")
        print("(Away) TEAM vs. (Home) TEAM")
        return None
    
    print(f"MULTIPLE GAME PREDICTIONS - Week {week}, {season}")
    print("="*60)
    print(f"Found {len(matches)} games to predict")
    print()
    
    # Get team features once for all predictions - Use 2025 Week 1 data if available
    try:
        team_features = predictor.create_team_features(weekly_data, 2025, week)
        
        # If no 2025 data available (early in season), fall back to end of 2024
        if not team_features:
            print("No 2025 data available yet, using 2024 season-end data...")
            team_features = predictor.create_team_features(weekly_data, 2024, 19)
            data_source = "2024 season-end"
        else:
            print(f"Using 2025 data through Week {week-1}...")
            data_source = f"2025 Week {week-1}"
        
        if not team_features:
            print("Could not create team features")
            return None
        
        # Debug: Show available teams and data source
        if show_details:
            print(f"Data Source: {data_source}")
            print(f"Available teams in data: {sorted(list(team_features.keys()))}")
            print()
            
    except Exception as e:
        print(f"Error loading team data: {e}")
        print("Make sure weekly_data is available.")
        return None
    
    all_predictions = []
    successful_predictions = 0
    
    # Track current time slot for headers
    current_day = None
    current_time_slot = None
    
    for i, (away_full, home_full) in enumerate(matches, 1):
        away_full = away_full.strip()
        home_full = home_full.strip()
        
        # Map to abbreviations
        away_team = team_mapping.get(away_full, away_full.split()[-1] if away_full.split() else away_full)
        home_team = team_mapping.get(home_full, home_full.split()[-1] if home_full.split() else home_full)
        
        # Clean up team names (remove extra spaces, etc.)
        away_team = away_team.strip().upper()
        home_team = home_team.strip().upper()
        
        if show_details:
            # Get game time info for header
            matchup_key = f"{away_team} @ {home_team}"
            if matchup_key in game_schedule:
                day, time = game_schedule[matchup_key]
                
                # Display time slot header if it's a new time slot
                if day != current_day or time.split()[0] != current_time_slot:
                    if current_day is not None:  # Add spacing between time slots
                        print()
                    print(f"{day.upper()}")
                    print(f"{time}")
                    print("-" * 50)
                    current_day = day
                    current_time_slot = time.split()[0]  # Get just the time part
            
            print(f"Game {i}: {away_team} @ {home_team} (Processing...)")
        
        try:
            # Check if teams exist in our data
            if away_team not in team_features or home_team not in team_features:
                if show_details:
                    print(f"   Missing team data for {away_team} or {home_team}")
                    available_teams = list(team_features.keys())[:10]  # Show first 10
                    print(f"   Available teams (sample): {available_teams}")
                continue
            
            # Create game features
            game_features = predictor.create_game_features(
                home_team, away_team, team_features,
                season, week, is_playoff=False, is_neutral=False
            )
            
            if game_features is None:
                if show_details:
                    print(f"   Could not create game features")
                continue
            
            # Make prediction
            game_df = pd.DataFrame([game_features])
            prediction_result = predictor.predict_games(game_df)
            
            # Extract results for this specific game
            home_win_prob = prediction_result['home_win_prob'].iloc[0]
            predicted_home_win = prediction_result['predicted_home_win'].iloc[0]
            high_confidence = prediction_result['high_confidence_bet'].iloc[0]
            
            # Determine winner for this specific game
            winner = home_team if predicted_home_win == 1 else away_team
            confidence = home_win_prob if predicted_home_win == 1 else (1 - home_win_prob)
            
            # Store prediction
            prediction_data = {
                'game_num': i,
                'away_team': away_team,
                'home_team': home_team,
                'matchup': f"{away_team} @ {home_team}",
                'predicted_winner': winner,
                'confidence': confidence,
                'home_win_prob': home_win_prob,
                'away_win_prob': 1 - home_win_prob,
                'high_confidence': high_confidence
            }
            
            all_predictions.append(prediction_data)
            successful_predictions += 1
            
            if show_details:
                # Get game time info
                matchup_key = f"{away_team} @ {home_team}"
                if matchup_key in game_schedule:
                    day, time = game_schedule[matchup_key]
                    print(f"   Game {i} Complete: {away_team} @ {home_team}")
                    print(f"   {day} at {time}")
                    print(f"   Winner: {winner} ({confidence:.1%})")
                    print(f"   Home: {home_win_prob:.1%} | Away: {(1-home_win_prob):.1%}")
                else:
                    print(f"   Game {i} Complete: {away_team} @ {home_team}")
                    print(f"   Winner: {winner} ({confidence:.1%})")
                    print(f"   Home: {home_win_prob:.1%} | Away: {(1-home_win_prob):.1%}")
                print()
        
        except Exception as e:
            if show_details:
                print(f"   Error: {e}")
                print()
            continue
    
    if not all_predictions:
        print("No successful predictions made")
        return None
    
    # Create summary DataFrame
    predictions_df = pd.DataFrame(all_predictions)
    
    # Summary statistics
    print(f"\nPREDICTION SUMMARY")
    print("="*40)
    print(f"Successful predictions: {successful_predictions}/{len(matches)}")
    print(f"Data source: {data_source}")
    
    # Show top confident predictions
    print(f"\nTOP CONFIDENT PREDICTIONS:")
    top_predictions = predictions_df.nlargest(3, 'confidence')
    for _, game in top_predictions.iterrows():
        print(f"   {game['matchup']} -> {game['predicted_winner']} ({game['confidence']:.1%})")
    
    return predictions_df

# Example: NFL Week 3 Complete Schedule (Actual 2025 Schedule)
week3_games = """
(Away) Miami Dolphins vs. (Home) Buffalo Bills
(Away) Atlanta Falcons vs. (Home) Carolina Panthers
(Away) Cincinnati Bengals vs. (Home) Minnesota Vikings
(Away) Houston Texans vs. (Home) Jacksonville Jaguars
(Away) Indianapolis Colts vs. (Home) Tennessee Titans
(Away) Las Vegas Raiders vs. (Home) Washington Commanders
(Away) Los Angeles Rams vs. (Home) Philadelphia Eagles
(Away) Green Bay Packers vs. (Home) Cleveland Browns
(Away) New York Jets vs. (Home) Tampa Bay Buccaneers
(Away) Pittsburgh Steelers vs. (Home) New England Patriots
(Away) Denver Broncos vs. (Home) Los Angeles Chargers
(Away) New Orleans Saints vs. (Home) Seattle Seahawks
(Away) Dallas Cowboys vs. (Home) Chicago Bears
(Away) Arizona Cardinals vs. (Home) San Francisco 49ers
(Away) Kansas City Chiefs vs. (Home) New York Giants
(Away) Detroit Lions vs. (Home) Baltimore Ravens
"""



# Quick prediction function for copy-paste game lists
def quick_predict_week2_games():
    """Quick function to predict Week 2 NFL games"""
    
    # Check if we have a trained predictor
    if 'predictor' in locals() or 'predictor' in globals():
        try:
            # Access predictor from global scope
            global predictor
            if predictor.final_model is not None:
                print("PREDICTING NFL WEEK 2 GAMES")
                print("="*50)
                
                results = predict_multiple_games(
                    predictor, 
                    week3_games, 
                    season=2025, 
                    week=3, 
                    show_details=True  # Show details to debug missing games
                )
                
                return results
            else:
                print("No trained model found. Train the model first.")
                return None
        except NameError:
            print("No predictor found. Train the model first.")
            return None
    else:
        print("No predictor found. Train the model first.")
        return None

def quick_predict_week1_games():
    """Quick function to predict Week 1 NFL games"""
    
    # Check if we have a trained predictor
    if 'predictor' in locals() or 'predictor' in globals():
        try:
            # Access predictor from global scope
            global predictor
            if predictor.final_model is not None:
                print("PREDICTING NFL WEEK 1 GAMES")
                print("="*50)
                
                results = predict_multiple_games(
                    predictor, 
                    season=2025, 
                    week=1, 
                    show_details=True  # Show details to debug missing games
                )
                
                return results
            else:
                print("No trained model found. Train the model first.")
                return None
        except NameError:
            print("No predictor found. Train the model first.")
            return None
    else:
        print("No predictor found. Train the model first.")
        return None

print("MULTIPLE GAME PREDICTION SYSTEM READY!")
print("\nAvailable Functions:")
print("  predict_multiple_games(predictor, games_text, season, week)")
print("  quick_predict_week3_games()  # NEW: Week 3 predictions")
print("\nUsage:")
print("  # Predict all Week 3 games:")
print("  results = quick_predict_week3_games()")
print("\n  # Custom game list:")
print("  my_games = '(Away) KC vs. (Home) BUF\\n(Away) DAL vs. (Home) PHI'")
print("  predict_multiple_games(predictor, my_games, 2025, 2)")

MULTIPLE GAME PREDICTION SYSTEM READY!

Available Functions:
  predict_multiple_games(predictor, games_text, season, week)
  quick_predict_week3_games()  # NEW: Week 3 predictions

Usage:
  # Predict all Week 3 games:
  results = quick_predict_week3_games()

  # Custom game list:
  my_games = '(Away) KC vs. (Home) BUF\n(Away) DAL vs. (Home) PHI'
  predict_multiple_games(predictor, my_games, 2025, 2)


## Step 6: Execute Week 3 Predictions

This cell runs predictions for all 16 NFL Week 3 games and provides:

**Two Output Formats:**

1. **Detailed Results**: Games listed chronologically with full date/time information
2. **Final Summary Table**: Clean, screenshot-friendly format showing:
   - All matchups in time order
   - Predicted winners (full team names)
   - Confidence percentages
   - Overall statistics

**Enhanced with 2025 Week 1 Data:** Uses the most current team performance data available to make accurate Week 2 predictions!

Perfect for sharing predictions or tracking model performance!

In [36]:
# RUN WEEK 3 GAME PREDICTIONS
print("EXECUTING NFL WEEK 3 PREDICTIONS")
print("="*50)

# Check if model is trained and ready
if 'predictor' in locals() and hasattr(predictor, 'final_model') and predictor.final_model is not None:
    print("Model is trained and ready!")
    print("Predicting all Week 3 games...")
    print()

    # Run predictions for all Week 3 games with concise output
    week3_results = predict_multiple_games(predictor, week3_games, season=2025, week=3, show_details=False)

    if week3_results is not None:
        print(f"Found {len(week3_results)} successful predictions out of 16 expected games")
        print("\nPREDICTED WINNERS:")
        print("="*50)

        # Week 3 game schedule with times (Eastern Time) - September 2025
        game_schedule = {
            'MIA @ BUF': ('Thursday, September 18', '8:15 PM ET'),
            'ATL @ CAR': ('Sunday, September 21', '1:00 PM ET'),
            'CIN @ MIN': ('Sunday, September 21', '1:00 PM ET'),
            'HOU @ JAX': ('Sunday, September 21', '1:00 PM ET'),
            'IND @ TEN': ('Sunday, September 21', '1:00 PM ET'),
            'LV @ WAS': ('Sunday, September 21', '1:00 PM ET'),
            'LA @ PHI': ('Sunday, September 21', '1:00 PM ET'),
            'GB @ CLE': ('Sunday, September 21', '1:00 PM ET'),
            'NYJ @ TB': ('Sunday, September 21', '1:00 PM ET'),
            'PIT @ NE': ('Sunday, September 21', '1:00 PM ET'),
            'DEN @ LAC': ('Sunday, September 21', '4:05 PM ET'),
            'NO @ SEA': ('Sunday, September 21', '4:05 PM ET'),
            'DAL @ CHI': ('Sunday, September 21', '4:25 PM ET'),
            'ARI @ SF': ('Sunday, September 21', '4:25 PM ET'),
            'KC @ NYG': ('Sunday, September 21', '8:20 PM ET'),
            'DET @ BAL': ('Monday, September 22', '8:15 PM ET')
        }

        
        # Display results in chronological order (not sorted by confidence)
        time_order = [
            'MIA @ BUF',
            'ATL @ CAR', 'CIN @ MIN', 'HOU @ JAX', 'IND @ TEN',
            'LV @ WAS', 'LA @ PHI', 'GB @ CLE', 'NYJ @ TB', 'PIT @ NE',
            'DEN @ LAC', 'NO @ SEA',
            'DAL @ CHI', 'ARI @ SF',
            'KC @ NYG',
            'DET @ BAL'

        ]
        
        for matchup in time_order:
            game_row = week3_results[week3_results['matchup'] == matchup]
            if not game_row.empty:
                game = game_row.iloc[0]
                time_info = game_schedule.get(matchup, ('', ''))
                day, time = time_info if time_info else ('', '')
                print(f"{day} {time}: {matchup} -> {game['predicted_winner']} ({game['confidence']:.1%})")
        
        print(f"\nSUMMARY:")
        print("-" * 30)
        print(f"Games Predicted: {len(week3_results)}")
        print(f"Average Confidence: {week3_results['confidence'].mean():.1%}")

        # FINAL RESULTS - Clean format for screenshots
        print(f"\n\nFINAL WEEK 3 PREDICTIONS")
        print("=" * 50)
        print(f"{'MATCHUP':<20} {'WINNER':<12} {'CONFIDENCE':<12}")
        print("-" * 50)
        
        # Team name mapping for display
        team_names = {
            'WAS': 'Commanders', 'GB': 'Packers', 'NYG': 'Giants', 'DAL': 'Cowboys',
            'SEA': 'Seahawks', 'PIT': 'Steelers', 'LA': 'Rams', 'TEN': 'Titans',
            'BUF': 'Bills', 'NYJ': 'Jets', 'NE': 'Patriots', 'MIA': 'Dolphins',
            'JAX': 'Jaguars', 'CIN': 'Bengals', 'SF': '49ers', 'NO': 'Saints',
            'CLE': 'Browns', 'BAL': 'Ravens', 'CHI': 'Bears', 'DET': 'Lions',
            'DEN': 'Broncos', 'IND': 'Colts', 'CAR': 'Panthers', 'ARI': 'Cardinals',
            'PHI': 'Eagles', 'KC': 'Chiefs', 'ATL': 'Falcons', 'MIN': 'Vikings',
            'TB': 'Buccaneers', 'HOU': 'Texans', 'LAC': 'Chargers', 'LV': 'Raiders'
        }
        
        # Display results in chronological order
        for matchup in time_order:
            game_row = week3_results[week3_results['matchup'] == matchup]
            if not game_row.empty:
                game = game_row.iloc[0]
                winner_abbr = game['predicted_winner']
                winner_name = team_names.get(winner_abbr, winner_abbr)
                print(f"{game['matchup']:<20} {winner_name:<12} {game['confidence']:.1%}")
        
        print("-" * 50)
        print(f"Total Games: {len(week3_results)} | Avg Confidence: {week3_results['confidence'].mean():.1%}")
        
        # Save results to variable for further analysis
        print(f"\nResults saved to 'week3_results' variable")

    else:
        print("ERROR: Failed to generate predictions")
        
else:
    print("ERROR: Model not trained yet!")
    print("\nPlease run Step 2 to train the model first")

EXECUTING NFL WEEK 3 PREDICTIONS
Model is trained and ready!
Predicting all Week 3 games...

MULTIPLE GAME PREDICTIONS - Week 3, 2025
Found 16 games to predict

No 2025 data available yet, using 2024 season-end data...

PREDICTION SUMMARY
Successful predictions: 16/16
Data source: 2024 season-end

TOP CONFIDENT PREDICTIONS:
   MIA @ BUF -> BUF (65.6%)
   LA @ PHI -> PHI (62.6%)
   LV @ WAS -> WAS (60.9%)
Found 16 successful predictions out of 16 expected games

PREDICTED WINNERS:
Thursday, September 18 8:15 PM ET: MIA @ BUF -> BUF (65.6%)
Sunday, September 21 1:00 PM ET: ATL @ CAR -> CAR (56.5%)
Sunday, September 21 1:00 PM ET: CIN @ MIN -> CIN (58.7%)
Sunday, September 21 1:00 PM ET: HOU @ JAX -> JAX (56.5%)
Sunday, September 21 1:00 PM ET: IND @ TEN -> IND (51.5%)
Sunday, September 21 1:00 PM ET: LV @ WAS -> WAS (60.9%)
Sunday, September 21 1:00 PM ET: LA @ PHI -> PHI (62.6%)
Sunday, September 21 1:00 PM ET: GB @ CLE -> GB (56.9%)
Sunday, September 21 1:00 PM ET: NYJ @ TB -> TB (54.5